In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = pd.read_csv('./train.csv')
mnist.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = mnist.values
data_x = data[:,1:]
data_x = data_x/255.0
data_x = np.reshape(data_x,(-1,28,28))
data_y = data[:,0]
data_y = tf.keras.utils.to_categorical(data_y)
print(data_x.shape)
print(data_y.shape)

(42000, 28, 28)
(42000, 10)


In [4]:
x_train = data_x[:30000,:]
x_test = data_x[30000:40000,:]
y_train = data_y[:30000]
y_test = data_y[30000:40000]

In [5]:
num_epochs = 40
n_features = 28
n_hidden = 128
n_timestamps = 28
n_classes = 10
lr = 0.05

In [6]:
with tf.variable_scope('input'):
    ## Placeholder for input vector with shape[batch, seq, embeddings]
    x = tf.placeholder(dtype=tf.float32,shape=[None,n_timestamps,n_features])
    ## Placeholder for input vector with shape[batch, num_classes]
    y = tf.placeholder(dtype=tf.float32, shape=[None, n_classes])

In [7]:
## Hidden Vector Initially
initial_hidden = x[:, 0, :]
initial_hidden = tf.matmul(initial_hidden, tf.ones([n_features, n_hidden]))
initial_hidden = tf.stack([initial_hidden, initial_hidden])

In [8]:
with tf.variable_scope('lstm_cell'):
    init = tf.contrib.layers.xavier_initializer()
    ## Forget Gate
    wf = tf.get_variable( 'wf', shape= [n_features,n_hidden],initializer= init )
    uf = tf.get_variable( 'uf', shape = [n_hidden,n_hidden], initializer=init )
    bf = tf.get_variable( 'bf', shape = [n_hidden], initializer=init )
    ## Input Gate
    wi = tf.get_variable( 'wi', shape= [n_features,n_hidden],initializer= init )
    ui = tf.get_variable( 'ui', shape = [n_hidden,n_hidden], initializer=init )
    bi = tf.get_variable( 'bi', shape = [n_hidden], initializer=init )
    ## New candidate Gate
    wc = tf.get_variable( 'wc', shape= [n_features,n_hidden],initializer= init )
    uc = tf.get_variable( 'uc', shape = [n_hidden,n_hidden], initializer=init )
    bc = tf.get_variable( 'bc', shape = [n_hidden], initializer=init )
    ## Output Gate
    wog = tf.get_variable( 'wog', shape= [n_features,n_hidden],initializer= init )
    uog = tf.get_variable( 'uog', shape = [n_hidden,n_hidden], initializer=init )
    bog = tf.get_variable( 'bog', shape = [n_hidden], initializer=init )   

In [9]:
with tf.variable_scope('dense'):
    wo = tf.get_variable( 'wo', shape= [n_hidden,n_classes],initializer= init )
    bo = tf.get_variable( 'bo', shape = [n_classes], initializer=init)

In [10]:
def graph_def(prev_hidden, x):
    previous_hidden_state, c_prev = tf.unstack(prev_hidden)
    i = tf.nn.sigmoid(tf.matmul(x,wi) + tf.matmul(previous_hidden_state,ui) + bi)
    f = tf.nn.sigmoid(tf.matmul(x,wf) + tf.matmul(previous_hidden_state,uf) + bf)
    o = tf.nn.sigmoid(tf.matmul(x,wog) + tf.matmul(previous_hidden_state,uog) + bog)
    c_ = tf.nn.tanh(tf.matmul(x,wc) + tf.matmul(previous_hidden_state,uc) + bc)
    c = tf.multiply(f,c_prev) + tf.multiply(i,c_)
    current_hidden_state = tf.multiply(o, tf.nn.tanh(c))
    return tf.stack([current_hidden_state,c])

In [11]:
with tf.variable_scope('history'):
    all_states = tf.Variable(tf.random_normal(shape=[n_timestamps,2,n_hidden]))

In [12]:
state_series = []
def get_state():
    prev_hidden = initial_hidden
    for ix in range(n_timestamps):
        cur_hidden = graph_def(prev_hidden,x[:,ix,:])
        prev_hidden = cur_hidden
    print(prev_hidden[0,:,:].shape)
    print(x[:,ix,:].shape)
    state_series.append(prev_hidden[0,:,:])
    return prev_hidden[0,:,:]

In [13]:
def get_output():
    hidden_state = get_state()
    output = tf.add(tf.matmul(hidden_state, wo), bo)
    return output

In [14]:
output = get_output()
prediction = tf.nn.softmax(output)

(?, 128)
(?, 28)


In [15]:
cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output,labels=y)
cost = tf.reduce_mean(cost)
saver = tf.train.Saver(max_to_keep=10)
summary = tf.summary.scalar('cost',cost)

In [16]:
train_writer = tf.summary.FileWriter('./logs/training_lstm')
train_writer.add_graph( tf.get_default_graph() )

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
# correct_prediction = (tf.argmax(prediction, 1) == tf.argmax(y, 1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))

In [18]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     last_ckpt = tf.train.latest_checkpoint('./lstm_weights')
#     saver.restore(sess,last_ckpt)
    for epoch in range(num_epochs):
        series,_,training_loss,train_summary,train_accuracy, prediction1 = sess.run([state_series,optimizer,cost,summary,accuracy,correct_prediction],feed_dict = {
            x:x_train,
            y:y_train
        })
        test_loss = sess.run(cost, feed_dict = {
            x:x_test,
            y:y_test
        })
        print('Epoch: '+str(epoch+1))
        print ( 'Training Loss is ' + str(training_loss) + '\nTesting loss is ' + str(test_loss) + '\n')
        print('Training Accuracy is ' + str(train_accuracy) + '\n')
        saver.save(sess, './lstm_weights/epoch_' + str(epoch + 1) + '.ckpt')
        print (" Model Saved \n ")
        print (" ----------- \n")

Epoch: 1
Training Loss is 2.3547463
Testing loss is 2.7500148

Training Accuracy is 0.098233335

 Model Saved 
 
 ----------- 

Epoch: 2
Training Loss is 2.7480898
Testing loss is 2.2088132

Training Accuracy is 0.1084

 Model Saved 
 
 ----------- 

Epoch: 3
Training Loss is 2.1996536
Testing loss is 2.4928765

Training Accuracy is 0.1623

 Model Saved 
 
 ----------- 

Epoch: 4
Training Loss is 2.494246
Testing loss is 2.4313967

Training Accuracy is 0.1988

 Model Saved 
 
 ----------- 

Epoch: 5
Training Loss is 2.4344206
Testing loss is 2.3355231

Training Accuracy is 0.19723333

 Model Saved 
 
 ----------- 

Epoch: 6
Training Loss is 2.3301485
Testing loss is 2.2991164

Training Accuracy is 0.1757

 Model Saved 
 
 ----------- 

Epoch: 7
Training Loss is 2.3042636
Testing loss is 2.0767746

Training Accuracy is 0.15113333

 Model Saved 
 
 ----------- 

Epoch: 8
Training Loss is 2.070943
Testing loss is 2.0009754

Training Accuracy is 0.23063333

 Model Saved 
 
 ----------- 

E

In [19]:
from keras.layers import LSTM, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [20]:
model = Sequential()
model.add(LSTM(128, return_sequences=False, input_shape = (28,28)) )
model.add(Dense(n_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               80384     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 81,674
Trainable params: 81,674
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [22]:
model.fit(x_train,y_train,epochs=10,batch_size=256,shuffle=True)

Epoch 1/10
30000/30000 [==============================] - 22s 718us/step - loss: 1.1617 - acc: 0.6056
Epoch 2/10
30000/30000 [==============================] - 20s 679us/step - loss: 0.4709 - acc: 0.8452
Epoch 3/10
30000/30000 [==============================] - 19s 648us/step - loss: 0.2912 - acc: 0.9077
Epoch 4/10
30000/30000 [==============================] - 24s 789us/step - loss: 0.2087 - acc: 0.9344
Epoch 5/10
30000/30000 [==============================] - 22s 724us/step - loss: 0.1608 - acc: 0.9501
Epoch 6/10
30000/30000 [==============================] - 23s 770us/step - loss: 0.1325 - acc: 0.9578
Epoch 7/10
30000/30000 [==============================] - 21s 704us/step - loss: 0.1117 - acc: 0.9650
Epoch 8/10
30000/30000 [==============================] - 19s 643us/step - loss: 0.0932 - acc: 0.9711
Epoch 9/10
30000/30000 [==============================] - 19s 646us/step - loss: 0.0811 - acc: 0.9749
Epoch 10/10
30000/30000 [==============================] - 19s 639us/step - loss: 